# Project 01 — Artificial Intelligence 2025/26
## Class Timetable (CSP Problem)

### 1 Introduction
>This project aims to develop an intelligent agent capable of generating class timetables for the undergraduate courses at the Polytechnic Institute of Cávado and Ave (IPCA). 
>Creating timetables is a complex and time-consuming task that involves multiple constraints related to professors, subjects, classrooms, and student groups. Traditionally, this process requires significant time and effort from the administrative team.
>In this context, the project intends to apply Artificial Intelligence techniques, specifically Constraint Satisfaction Problems (CSP), to automate the creation of viable and optimized timetables, respecting the imposed conditions and minimizing conflicts.
>The project will be implemented in Python, using the python-constraint library, and fully documented in a Jupyter Notebook, as per the guidelines of the Artificial Intelligence (AI) course.

**Group members:**
- Pedro Ribeiro — student number 27960
- Ricardo Fernandes — student number 27961  
- Carolina Branco — student number 27983
- João Barbosa — student number 27964
- Diogo Abreu — student number 27975

---

### 1. Goal Formulation
- Clearly describe the objective of the agent.  
- Identify the limitations (e.g., teacher availability, number of classrooms, etc.).  
- Explain what kind of results are expected.

---

### 2. Problem Formulation (CSP)
Explain how the problem was modeled:  
- **Variables:** (e.g., class, course, teacher, room, time slot)  
- **Domain:** (possible values for each variable)  
- **Constraints:**  
  - Hard constraints  
  - Soft constraints  
- **Heuristics applied:** 

---

### 3. Implementação (Agente)


In [1]:
from constraint import *
from datasets.constants import *
import itertools 

#1. Import datasets

dataset_path = "datasets/timetable_dataset.txt"

courses_assigned_to_classes = {}
courses_assigned_to_lecturers = {}
timeslot_restrictions = {}
roomrestrictions = {}
online_classes = {}

with open(dataset_path, "r") as f:
    lines = f.readlines()

reading_section = None

for line in lines:
    line = line.strip()
    if not line:
        continue
    if line.startswith("#cc"):
        reading_section = "cc"
        continue
    elif line.startswith("#dsd"):
        reading_section = "dsd"
        continue
    elif line.startswith("#tr"):
        reading_section = "tr"
        continue
    elif line.startswith("#rr"):
        reading_section = "rr"
        continue
    elif line.startswith("#oc"):
        reading_section = "oc"
        continue
    elif line.startswith("#"):
        reading_section = None
        continue

    if reading_section == "cc":
        parts = line.split()
        class_name = parts[0]
        courses = parts[1:]
        courses_assigned_to_classes[class_name] = courses
    elif reading_section == "dsd":
        parts = line.split()
        teacher_name = parts[0]
        courses = parts[1:]
        courses_assigned_to_lecturers[teacher_name] = courses
    elif reading_section == "tr":
        parts = line.split()
        teacher = parts[0]
        courses = list(map(int, parts[1:]))
        timeslot_restrictions[teacher] = courses
    elif reading_section == "rr":
        parts = line.split()
        course = parts[0]
        room = parts[1:]
        roomrestrictions[course] = room
    elif reading_section == "oc":
        parts = line.split()
        course = parts[0]
        lesson_week_index = parts[1:]
        online_classes[course] = lesson_week_index


print("Class courses:", courses_assigned_to_classes)
print("Teachers courses:", courses_assigned_to_lecturers)
print("Teacher unavailable slots:", timeslot_restrictions)
print("Room restrictions:", roomrestrictions)
print("Online classes:", online_classes)

#2. Create CSP

problem = Problem()
rooms = ["Lab01","Room1", "Room2"]

lessons_per_course = 2
slots = list(range(1, 21))

for class_name, courses in courses_assigned_to_classes.items():
    for course in courses:
        for lesson in range(1, lessons_per_course + 1):
            problem.addVariable(f"{course}_{lesson}_slot", slots)
            if course in roomrestrictions:
                problem.addVariable(f"{course}_{lesson}_room", [roomrestrictions[course]])
            else:
                problem.addVariable(f"{course}_{lesson}_room", rooms)

for class_name, courses in courses_assigned_to_classes.items():
    lesson_vars = [f"{course}_{lesson}_slot" for course in courses for lesson in range(1, lessons_per_course + 1)]
    problem.addConstraint(no_consecutive_slots, lesson_vars)

for room in rooms:
    room_slot_vars = []
    for class_name, courses in courses_assigned_to_classes.items():
        for course in courses:
            for lesson in range(1, lessons_per_course + 1):
                room_slot_vars.append((f"{course}_{lesson}_room", f"{course}_{lesson}_slot"))
    
    var_names = []
    for pair in room_slot_vars:
        var_names.extend(pair)
    
    problem.addConstraint(lambda *args, r=room: room_no_consecutive(r, *args), var_names)


print("Variáveis do CSP:")
for var_name in problem._variables:
    print(var_name, "→ domínio:", problem._variables[var_name])

Class courses: {'t01': ['UC11', 'UC12', 'UC13', 'UC14', 'UC15'], 't02': ['UC21', 'UC22', 'UC23', 'UC24', 'UC25'], 't03': ['UC31', 'UC32', 'UC33', 'UC34', 'UC35']}
Teachers courses: {'jo': ['UC11', 'UC21', 'UC22', 'UC31'], 'mike': ['UC12', 'UC23', 'UC32'], 'rob': ['UC13', 'UC14', 'UC24', 'UC33'], 'sue': ['UC15', 'UC25', 'UC34', 'UC35']}
Teacher unavailable slots: {'mike': [13, 14, 15, 16, 17, 18, 19, 20], 'rob': [1, 2, 3, 4], 'sue': [9, 10, 11, 12, 17, 18, 19, 20]}
Room restrictions: {'UC14': ['Lab01'], 'UC22': ['Lab01']}
Online classes: {'UC21': ['2'], 'UC31': ['2']}
Variáveis do CSP:
UC11_1_slot → domínio: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
UC11_1_room → domínio: ['Lab01', 'Room1', 'Room2']
UC11_2_slot → domínio: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
UC11_2_room → domínio: ['Lab01', 'Room1', 'Room2']
UC12_1_slot → domínio: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
UC12_1_room → domínio